In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

import time

from PyDemoMomentumopt import MotionPlanner
from src.momentumopt.kinoptpy.momentum_kinematics_optimizer import MomentumKinematicsOptimizer

def replay_kinematic_plan():
    for ks in motion_planner.kin_optimizer.kinematics_sequence.kinematics_states:
        q = ks.robot_posture.generalized_joint_positions
        motion_planner.kin_optimizer.robot.display(np.matrix(q).T)
        time.sleep(0.1)

In [ ]:
# cfg_file = '../config/SAB_Demos/cfg_quadruped_forward_jump.yaml'
# cfg_file = '../config/cfg_demo01_walk_short.yaml'
# cfg_file = '../config/cfg_quadruped_lift_leg_momentum.yaml'
cfg_file = '../config/cfg_quadruped_lift_leg_momentum_rotated.yaml'

mkin_opt = MomentumKinematicsOptimizer()

motion_planner = MotionPlanner(cfg_file, KinOpt=MomentumKinematicsOptimizer)
# optimized_kin_plan, optimized_motion_eff, optimized_dyn_plan, planner_setting, time_vector = motion_planner.optimize_motion()

In [ ]:
# Dynamic optimization
# motion_planner.optimize_dynamics(0)
# motion_planner.optimize_kinematics(0)
# motion_planner.plot_centroidal()
# motion_planner.replay_kinematics()

motion_planner.optimize_dynamics(1)
motion_planner.optimize_kinematics(1)
motion_planner.plot_centroidal()
motion_planner.replay_kinematics()

In [ ]:
import pinocchio as se3

robot = motion_planner.kin_optimizer.robot
q = motion_planner.kin_optimizer.q_init
dq = motion_planner.kin_optimizer.dq_init

quad = se3.Quaternion(se3.rpy.rpyToMatrix(np.matrix([0.0, 0, np.pi/4]).T))
# quad = se3.Quaternion(se3.rpy.rpyToMatrix(np.matrix([0, 0, 0]).T))
q[3:7] = quad.coeffs()

robot.robot.forwardKinematics(q, dq)
robot.robot.framesForwardKinematics(q)

robot.display(q)

for eff in [robot.robot.data.oMf[idx].translation for idx in  motion_planner.kin_optimizer.inv_kin.endeff_ids]:
    print(eff.T)

In [ ]:
# Test the frame orientation for data.Ag and data.hg
# -> Both are expressed wrt. to the inertia frame.

import pinocchio as se3

q = motion_planner.kin_optimizer.q_init.copy()
robot = motion_planner.kin_optimizer.robot

q[3:6] = 0.
q[6] = 1.

robot.display(q)
dq = motion_planner.kin_optimizer.dq_init

# time.sleep(0.5)

dq[0] = 0.5
dq[3] = 0.5

q_next = se3.integrate(robot.model, q, dq)
robot.display(q_next)

robot.robot.centroidalMomentum(q, dq).linear
print(robot.robot.data.Ag.dot(dq))

print(robot.robot.data.hg.linear)
print(robot.robot.data.hg.angular)

In [ ]:
%load_ext autoreload
%autoreload 2


mkin_opt.initialize(motion_planner.planner_setting, 50)

mkin_opt.optimize(motion_planner.ini_state, 
                  motion_planner.contact_plan.contactSequence(),
                  motion_planner.dyn_optimizer.dynamicsSequence())

for q in mkin_opt.q_kin:
    mkin_opt.robot.display(np.matrix(q).T)
    time.sleep(mkin_opt.dt)

In [ ]:
# Create trajectories for the endeffectors

In [ ]:

    
fig, axes = plt.subplots(3, 1, figsize=(6, 8), sharex=True)

dynseq = {'com': mkin_opt.com_dyn, 'lmom': mkin_opt.lmom_dyn, 'amom': mkin_opt.amom_dyn}
kinseq = {'com': mkin_opt.com_kin, 'lmom': mkin_opt.lmom_kin, 'amom': mkin_opt.amom_kin}

for i, (ax, prop) in enumerate(zip(axes, ['com', 'lmom', 'amom'])):
    data_dyn = dynseq[prop][:it_opt]
    data_kin = kinseq[prop][:it_opt]

    for dyn, kin, label in zip(data_dyn.T, data_kin.T, ['{}_{}'.format(prop, d) for d in ['x', 'y', 'z']]):
        line = ax.plot(dyn, label=label, alpha=0.75)[0]
        ax.plot(kin, '--', color=line.get_color())[0]

    ax.legend()
    ax.grid(True)

fig.suptitle('Centroidal info for dyn (-) and kin (--)')
fig.tight_layout(rect=[0, 0, 1., 0.95])